In [150]:
# For this notebook your env will need pandas, openpyxl, msal installed via pip or from source if you prefer.  The client_secret.txt file and the empty IOC xlsx files should be in the same folder the notebook is running from or should have a specified filepath.

import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
import os
import requests
import json

# Read the client secret from a text file
with open("client_secret.txt", "r") as f:
    client_secret = f.read().strip()

# Set the client secret as an environment variable
os.environ["CLIENT_SECRET"] = client_secret

from msal import ConfidentialClientApplication

# Azure AD application credentials
client_id = "d19c28a3-d5a3-4ec8-952f-9c21915ee83d"
# If you are not using a secure string or key vault, you will need to un-comment the line below and add the secret there.
# client_secret = ""
tenant_id = "eb81c4c9-2546-43f2-8c43-9c2295af4b88"


# Create a ConfidentialClientApplication object
app = ConfidentialClientApplication(
    client_id=client_id,
    client_credential=client_secret,
    authority=f"https://login.microsoftonline.com/{tenant_id}",
)

# Get a token from Azure AD
result = None
scopes = ["https://graph.microsoft.com/.default"]
result = app.acquire_token_silent(scopes=scopes, account=None)

if not result:
    result = app.acquire_token_for_client(scopes=scopes)

# Get the access token
access_token = result["access_token"]

# Print the access token
print("Access Token:", access_token)

Access Token: eyJ0eXAiOiJKV1QiLCJub25jZSI6InhqOFBuMHRIdDgzYW5UWEdZR1RNUUZnaWRLUkZEZk1GNVdYOUQya1RIN0UiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC9lYjgxYzRjOS0yNTQ2LTQzZjItOGM0My05YzIyOTVhZjRiODgvIiwiaWF0IjoxNjg2OTM0MDYwLCJuYmYiOjE2ODY5MzQwNjAsImV4cCI6MTY4NjkzNzk2MCwiYWlvIjoiRTJZQWdwS2dKUmN5QWdYRnRSVmFYMnBaYmdVQSIsImFwcF9kaXNwbGF5bmFtZSI6IlRJLUFQSSIsImFwcGlkIjoiZDE5YzI4YTMtZDVhMy00ZWM4LTk1MmYtOWMyMTkxNWVlODNkIiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvZWI4MWM0YzktMjU0Ni00M2YyLThjNDMtOWMyMjk1YWY0Yjg4LyIsImlkdHlwIjoiYXBwIiwib2lkIjoiYzZhMjYxYWUtYTAwZi00YjYwLWEyODYtODYxNzM4ODNiZjE1IiwicmgiOiIwLkFSTUF5Y1NCNjBZbDhrT01RNXdpbGE5TGlBTUFBQUFBQUFBQXdBQUFBQUFBQUFBVEFBQS4iLCJyb2xlcyI6WyJTZWN1cml0eUFsZXJ0LlJlYWRXcml0ZS5BbGwiLCJTZWN1cml0eUFjdGlvbnMuUmVhZFdyaXRlLkFsbCIsIlNlY3VyaXR5RXZlbnRzLlJlYWQuQWxsIiwiU2VjdXJpdHlBbGVyd

Reputation

This block will accept domains/IPs in an array and pass them through to the reputation endpoint for analysis.  The resulting dataframe is setup to provide data from 2 separate JSON normalizations.  If you display a single dataframe from the list below, it will have nested JSON which will not display nicely so we combine the two for an easier viewing experience.

In [151]:
pd.set_option('display.max_colwidth', 0)

suspect_iocs = ["162.33.178.162", "vanguard.om"]
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}


for ip in suspect_iocs:
    services = f"https://graph.microsoft.com/beta/security/threatIntelligence/hosts/"+(ip)+'/reputation'
    responseS = requests.get(services, headers=headers)
    dataS = responseS.json()
    dataframeS = pd.json_normalize(dataS)
    dataframeSi = pd.json_normalize(dataS['rules'])
    display(dataframeS[['@odata.context','score', 'classification']], dataframeSi)



,@odata.context,score,classification
0,https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hosts('162.33.178.162')/reputation/$entity,100,malicious


,name,description,severity,relatedDetailsUrl
0,MDTI Intel Article,Storm-0216 deploys Cactus ransomware beginning April 2023,high,https://ti.defender.microsoft.com/article/fb337484
1,Microsoft Blocklist (WatchList),"Please read the linked article(s) for more information on this threat. [""https://ti.defender.microsoft.com/article/fb337484""]",high,None
2,ASN,Infrastructure hosted by this ASN frequently exhibits suspicious behavior,medium,None
3,SSL certificate age,New certificates can be a sign of recent change in infrastructure,low,None
4,SSL certificate self-signed,Self-signed certificates lack verified ownership and may not be trustworthy,low,None


,@odata.context,score,classification
0,https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hosts('vanguard.om')/reputation/$entity,100,malicious


,name,description,severity,relatedDetailsUrl
0,MDTI Intel Article,Volt Typhoon targets US critical infrastructure with living-off-the-land techniques,high,https://ti.defender.microsoft.com/article/223703a9
1,Microsoft Blocklist (WatchList),"Please read the linked article(s) for more information on this threat. [""https://ti.defender.microsoft.com/article/223703a9""]",high,None


**Articles**

In the above example we see that an article has been written to provide more context around the IOC we're looking up.  Below we can grab the article content for faster review and also below this block we can extract the IOCs which are attributed to this campaign.

In [152]:
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

article = '223703a9'

services = f"https://graph.microsoft.com/beta/security/threatIntelligence/articles/"+(article)
responseS = requests.get(services, headers=headers)
dataA = responseS.json()
dataframeA = pd.json_normalize(dataA)
display(dataframeA)

@odata.context  \
0  https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/articles/$entity   

         id          createdDateTime       lastUpdatedDateTime  \
0  223703a9  2023-05-24T18:57:17.35Z  2023-06-12T22:58:36.387Z   

                                                                                 title  \
0  Volt Typhoon targets US critical infrastructure with living-off-the-land techniques   

   isFeatured  \
0  True         

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    tags  \
0  [Volt Typhoon, T1593.002 - Search Engines, T1584.005 - Botnet, T1586.002 - Email Accounts, T1059.003 - Windows Command Shell, T1059.001 - PowerShell, T1059.008 - Network Device CLI, T1047 - Windows Management Instrumentation, T1078.002 - Domain Accounts, T1070.001 - Clear Windows Event Logs, T1070.006 - Timestomp, T1070.007 - Clear Network Connection History and Configurations, T1003.001 - LSASS Memory, T1497.001 - System Checks, T1018 - Remote System Discovery, T1082 - System Information Discovery, T1602 - Data from Configuration Repository, T1005 - Data from Local System, T1074 - Data Staged, T1090.001 - Internal Proxy]   

                                              imageUrl  \
0  https://cdn-riq-ti.azureedge.net/AbstractCoding.png   

                                                                                                                                                                                                                                                                                                                                                                                                                                       summary.content  \
0  Chinese state-sponsored actor Volt Typhoon is using living-off-the-land binaries and other stealthy techniques to target US critical infrastructure, conduct espionage, and dwell in compromised environments. This campaign focuses on organizations in Guam and elsewhere in the United States, in the communications, manufacturing, utility, transportation, construction, maritime, government, information technology, and education sectors.   

  summary.format  \
0  markdown        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

**Article Indicators**

This block will pull the indicators and types from an article, this could be passed to an array or uploaded into Sentinel's TI blade for further investigation for example.  The return is clipped down to two columns, you can explore all of the columns by removing the [[['artifact.id', 'artifact.kind']]] on the last line.

In [55]:
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

article = '9b2c8898'

services = f"https://graph.microsoft.com/beta/security/threatIntelligence/articles/"+(article)+"/indicators"
responseS = requests.get(services, headers=headers)
dataA = responseS.json()
dataframeA = pd.json_normalize(dataA['value'])
display(dataframeA[['artifact.id', 'artifact.kind']])

,artifact.id,artifact.kind
0,e0c6567d2393d093a74220df479f996b40a59200bdb1f04b83339d51ad559c9c,hash_sha256
1,f971a3ed94dd16241597c16d86da9b9fc1cdf715a0be5cb4240009a80aa5a5af,hash_sha256
2,9ede138e3ecdf7d3bb9f614124cbfb0edee76fa1a4564ec1db32d25441fe552d,hash_sha256
3,53e4bfd27474f6e4829ac4d625d3d914452456baf5da2c1c51e2e6df35ab634a,hash_sha256
4,b59686a8ead6491bce8633f5e774fd5c3f52bc0f733cd168f113729dfdfa702f,hash_sha256
5,fbb221ee4b17929bddc95beac7d2736709cf1a5c161c3139a1cd90c3f2044420,hash_sha256
6,a68ecce02b94316742071ce77cbcfb19f582eddf610c70ddf02112022117a22f,hash_sha256
7,e8ff3d2fe2316848c079947789c3063b006678089770646cd1ef4e226d95824e,hash_sha256
8,8fc6522624a7727e8d4b61a3e87ff77a3e77a4331b72d4f363c755aa33e74657,hash_sha256
9,993c7135767254b07753d2cabbbc10d44b96345d8225118e1750c09011a62371,hash_sha256


In [154]:
pd.set_option('display.max_colwidth', 0)
group_name = "Volt Typhoon"

group_lookup = f"https://graph.microsoft.com/beta/security/threatIntelligence/intelProfiles?$search="+(group_name)+""
responseGr = requests.get(group_lookup, headers=headers)
dataGr = responseGr.json()
dataframeGr = pd.json_normalize(dataGr['value'])
    #dataframeT = dataframeT.drop(['id'], axis=1)     
display(dataframeGr)

id   kind  \
0  8fe93ebfb3a03fb94a92ac80847790f1d6cfa08f57b2bcebfad328a5c3e762cb  actor   

          title   firstActiveDateTime                              aliases  \
0  Volt Typhoon  2023-05-10T00:00:00Z  [VANGUARD PANDA, BRONZE SILHOUETTE]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      targets  \
0  [Communications Infrastructure: Satellite, Communications Infrastructure: Wireless, Communications Infrastructure: Wireline, Critical Manufacturing: Electrical Equipment, Appliance, and Component Manufacturing, Defense Industrial Base, Digital, Print and Broadcast Media, Education: Higher Education, Government Agencies & Services: General Public Services - Local, Information Technology: IT Products & Services, Transportation Systems: Mass Transit and Passenger Rail, Water & Wastewater Systems: Drinking Water, Water & Wastewater Systems: Wastewater]   

  sponsorStates  \
0  []             

                                                                                                                                                                                                                                                                                                                                                             summary.content  \
0  The actor that Microsoft tracks as Volt Typhoon is a nation-state activity group based out of China. Volt Typhoon is known to primarily target the United States and the manufacturing, utility, transportation, construction, maritime, government, information technology, and education sectors. Volt Typhoon focuses on espionage, data theft, and credential access.   

  summary.format  \
0  markdown        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               description.content  \
0  ## Snapshot\r\nThe actor that Microsoft tracks as Volt Typhoon is a nation-state activity group based out of China. Volt Typhoon is known to primarily target the United States and the manufacturing, utility, transportation, construction, maritime, government, information technology, and education sectors. Volt Typhoon focuses on espionage, data theft, and credential access.\r\n\r\n## Targeting Details\r\nVolt Typhoon has been active since mid-2021 and has targeted critical infrastructure organizations in Guam and elsewhere in the United States. In this campaign, the affected organizations span the communications, manufacturing, utility, transportation, const

In [157]:
group_id = "8fe93ebfb3a03fb94a92ac80847790f1d6cfa08f57b2bcebfad328a5c3e762cb"

groupIOC_lookup = f"https://graph.microsoft.com/beta/security/threatIntelligence/intelProfiles/"+(group_id)+"/indicators"
responseGroupIOC = requests.get(groupIOC_lookup, headers=headers)
dataGroupIOC = responseGroupIOC.json()
dataframeGroupIOC = pd.json_normalize(dataGroupIOC['value'])
#dataframeGroupIOC = dataframeGroupIOC.drop(['id', 'artifact.@odata.type'], axis=1)     
display(dataframeGroupIOC)

,id,source,firstSeenDateTime,lastSeenDateTime,artifact.@odata.type,artifact.id,artifact.kind,artifact.value
0,7da8c3ef-f994-0e9c-e60e-298dfea68d1d,microsoftDefenderThreatIntelligence,2022-08-12T18:03:31Z,2023-06-15T11:00:36.013Z,#microsoft.graph.security.unclassifiedArtifact,7da8c3ef-f994-0e9c-e60e-298dfea68d1d,hash_sha256,BAEFFEB5FDEF2F42A752C65C2D2A52E84FB57EFC906D981F89DD518C314E231C
1,26f45965-8ac8-0d11-6408-213ff3c28bb2,microsoftDefenderThreatIntelligence,2022-11-28T19:57:23Z,2023-05-24T19:27:41.4Z,#microsoft.graph.security.unclassifiedArtifact,26f45965-8ac8-0d11-6408-213ff3c28bb2,hash_sha256,B4F7C5E3F14FB57BE8B5F020377B993618B6E3532A4E1EB1EAE9976D4130CC74
2,b8907c8b-c0ea-937b-71c1-feeab309ef38,microsoftDefenderThreatIntelligence,2022-12-09T23:25:41Z,2023-05-24T19:27:41.407Z,#microsoft.graph.security.unclassifiedArtifact,b8907c8b-c0ea-937b-71c1-feeab309ef38,hash_sha256,D6AB36CB58C6C8C3527E788FC9239D8DCC97468B6999CF9CCD8A815C8B4A80AF
3,3de6c9f8-49d7-bb3f-174e-14587fc9be30,microsoftDefenderThreatIntelligence,2022-12-09T23:25:41Z,2023-05-24T19:27:41.427Z,#microsoft.graph.security.unclassifiedArtifact,3de6c9f8-49d7-bb3f-174e-14587fc9be30,hash_sha256,9DD101CAEE49C692E5DF193B236F8D52A07A2030EED9BD858ED3AACCB406401A
4,9d7507a7-3156-a330-656e-cf365d009ddb,microsoftDefenderThreatIntelligence,2022-12-09T23:25:41Z,2023-05-24T19:27:40.947Z,#microsoft.graph.security.unclassifiedArtifact,9d7507a7-3156-a330-656e-cf365d009ddb,hash_sha256,7939F67375E6B14DFA45EC70356E91823D12F28BBD84278992B99E0D2C12ACE5
5,2af92a1c-dee6-4a97-cf26-d4e72b1fc3dc,microsoftDefenderThreatIntelligence,2022-12-09T23:25:41Z,2023-06-15T11:01:27.927Z,#microsoft.graph.security.unclassifiedArtifact,2af92a1c-dee6-4a97-cf26-d4e72b1fc3dc,hash_sha256,C0FC29A52EC3202F71F6378D9F7F9A8A3A10EB19ACB8765152D758ADED98C76D
6,47fcd4f9-659a-0b51-0bea-468d4a47f98a,microsoftDefenderThreatIntelligence,2022-12-09T23:25:41Z,2023-05-24T19:27:41.307Z,#microsoft.graph.security.unclassifiedArtifact,47fcd4f9-659a-0b51-0bea-468d4a47f98a,hash_sha256,93CE3B6D2A18829C0212542751B309DACBDC8C1D950611EFE2319AA715F3A066
7,bbfc3795-1882-cc62-5861-84713915278c,microsoftDefenderThreatIntelligence,2022-12-09T23:25:41Z,2023-05-24T19:27:41.307Z,#microsoft.graph.security.unclassifiedArtifact,bbfc3795-1882-cc62-5861-84713915278c,hash_sha256,4B0C4170601D6E922CF23B1CAF096BBA2FADE3DFCF92F0AB895A5F0B9A310349
8,e37cfacb-05be-9efd-9191-27a111a05249,microsoftDefenderThreatIntelligence,2022-12-09T23:25:41Z,2023-05-24T19:27:40.917Z,#microsoft.graph.security.unclassifiedArtifact,e37cfacb-05be-9efd-9191-27a111a05249,hash_sha256,450437D49A7E5530C6FB04DF2E56C3AB1553ADA3712FAB02BD1EEB1F1ADBC267
9,0466fe03-78d5-8009-5317-b7235162cf05,microsoftDefenderThreatIntelligence,2022-12-12T14:13:03Z,2023-06-15T11:02:02.817Z,#microsoft.graph.security.unclassifiedArtifact,0466fe03-78d5-8009-5317-b7235162cf05,hash_sha256,389A497F27E1DD7484325E8E02BBDF656D53D5CF2601514E9B8D8974BEFDDF61


**Web Components**

Web Components are vital to investigations, they allow us to map the services in use by our adversarial infrastructure.  This block will take an array of domains/IPs and return a cleaned up list of the components observed on each.  The .pop function is to remove the request ID from the returned information which isn't relevant to your investigation.

In [158]:
pd.set_option('display.max_colwidth', 0)

suspect_iocs = ["64.52.80.63","5.252.177.180","193.149.189.224","64.190.113.172","64.52.80.209","65.109.31.190","45.128.156.46","84.252.94.184","192.240.116.106","5.255.100.206"]
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
for ip in suspect_iocs:

    services = f"https://graph.microsoft.com/beta/security/threatIntelligence/hosts/"+(ip)+"/components"
    responseS = requests.get(services, headers=headers)
    dataP = responseS.json()
    dataframeP = pd.json_normalize(dataP['value'])
    dataframeP.pop('id')
    display(dataframeP)

,firstSeenDateTime,lastSeenDateTime,name,version,category,host.id
0,2023-01-30T17:48:38Z,2023-06-14T22:49:10Z,Ubuntu,,Operating System,64.52.80.63
1,2022-06-27T14:58:24Z,2023-06-14T22:49:10Z,SSH,2.0,Remote Access,64.52.80.63
2,2023-05-02T00:26:35Z,2023-06-14T22:49:10Z,OpenSSH,7.6p1,Remote Access,64.52.80.63
3,2023-05-04T08:24:13.892Z,2023-06-14T16:16:53Z,nginx,1.20.0,Server,64.52.80.63
4,2023-05-04T16:54:42Z,2023-06-13T03:54:09Z,MySQL,8.0.33,Data Store,64.52.80.63
5,2023-05-25T04:21:41.606Z,2023-05-25T04:21:43.092Z,jQuery,3.6.0,JavaScript Library,64.52.80.63
6,2023-05-25T04:21:41.606Z,2023-05-25T04:21:41.606Z,Google Tag Manager,,Analytics Service,64.52.80.63
7,2023-05-25T04:21:41.606Z,2023-05-25T04:21:41.606Z,Google Analytics,,Analytics Service,64.52.80.63
8,2023-01-30T17:48:38Z,2023-04-18T02:54:40Z,OpenSSH,8.9p1,Remote Access,64.52.80.63
9,2022-08-03T05:56:18Z,2023-01-22T18:18:03Z,Windows Remote Desktop,,Remote Access,64.52.80.63


,firstSeenDateTime,lastSeenDateTime,name,version,category,host.id
0,2022-12-02T01:44:52Z,2023-06-14T22:43:32Z,OpenSSH,8.2p1,Remote Access,5.252.177.180
1,2022-05-13T00:40:04Z,2023-06-14T22:43:32Z,Ubuntu,,Operating System,5.252.177.180
2,2021-05-24T00:54:46Z,2023-06-14T22:43:32Z,SSH,2.0,Remote Access,5.252.177.180
3,2023-04-24T19:37:32Z,2023-06-13T10:59:34Z,nginx,1.18.0,Server,5.252.177.180
4,2022-09-21T21:20:09Z,2022-10-19T12:28:54Z,mod_perl,2.0.12,Server Module,5.252.177.180
5,2022-09-21T21:20:09Z,2022-10-19T12:28:54Z,Apache,2.4.53,Server,5.252.177.180
6,2022-09-21T21:20:09Z,2022-10-19T12:28:54Z,PHP,7.4.29,Server Module,5.252.177.180
7,2022-09-21T21:20:09Z,2022-10-19T12:28:54Z,PHP,7.4.29,Framework,5.252.177.180
8,2022-09-21T21:20:09Z,2022-10-19T12:28:54Z,Unix,,Operating System,5.252.177.180
9,2022-09-21T21:20:09Z,2022-10-19T12:28:54Z,OpenSSL,1.1.1o,Server Module,5.252.177.180


,firstSeenDateTime,lastSeenDateTime,name,version,category,host.id
0,2022-11-03T13:45:42Z,2023-06-14T22:55:10Z,Ubuntu,,Operating System,193.149.189.224
1,2022-11-06T05:09:03Z,2023-06-14T22:55:10Z,OpenSSH,8.9p1,Remote Access,193.149.189.224
2,2022-11-03T13:45:42Z,2023-06-14T22:55:10Z,SSH,2.0,Remote Access,193.149.189.224
3,2023-05-03T20:08:14Z,2023-06-12T05:25:16Z,Windows,,Operating System,193.149.189.224
4,2023-05-03T20:08:14Z,2023-06-12T05:25:16Z,Windows Remote Desktop,,Remote Access,193.149.189.224
5,2023-03-16T18:06:29Z,2023-03-20T15:32:26Z,OpenSSH,7.9p1,Remote Access,193.149.189.224
6,2023-03-17T10:38:20.487Z,2023-03-17T10:42:42.289Z,Debian,,Operating System,193.149.189.224
7,2023-03-17T10:38:20.487Z,2023-03-17T10:42:42.289Z,Debian,,Server,193.149.189.224
8,2023-03-17T10:38:20.487Z,2023-03-17T10:42:42.289Z,Apache,2.4.38,Server,193.149.189.224
9,2022-12-05T16:18:52Z,2023-02-14T05:04:21Z,OpenSSH,7.4,Remote Access,193.149.189.224


,firstSeenDateTime,lastSeenDateTime,name,version,category,host.id
0,2022-05-24T18:03:56Z,2023-06-14T22:49:24Z,OpenSSH,7.4,Remote Access,64.190.113.172
1,2022-05-24T18:03:56Z,2023-06-14T22:49:24Z,SSH,2.0,Remote Access,64.190.113.172
2,2023-05-21T12:30:31Z,2023-06-14T16:20:50Z,WordPress,6.2.2,CMS,64.190.113.172
3,2023-03-11T00:29:14Z,2023-06-14T16:20:50Z,nginx,1.20.1,Server,64.190.113.172
4,2023-03-31T13:23:32Z,2023-06-14T16:20:50Z,All in One SEO Pack,4.3.4.1,Wordpress Plugin,64.190.113.172
5,2023-03-11T00:29:14Z,2023-06-14T16:20:50Z,PHP,8.0.28,Framework,64.190.113.172
6,2023-05-25T04:20:57.457Z,2023-05-25T04:20:57.457Z,WordPress,,CMS,64.190.113.172
7,2023-05-25T04:20:43.966Z,2023-05-25T04:20:43.966Z,Google,,Ad Network,64.190.113.172
8,2023-05-25T04:20:43.966Z,2023-05-25T04:20:43.966Z,jQuery,,JavaScript Library,64.190.113.172
9,2023-05-25T04:20:43.966Z,2023-05-25T04:20:43.966Z,Google Analytics,,Analytics Service,64.190.113.172


,firstSeenDateTime,lastSeenDateTime,name,version,category,host.id
0,2022-10-08T18:28:13Z,2023-06-10T20:28:14Z,Windows Remote Desktop,,Remote Access,64.52.80.209
1,2022-10-08T18:28:13Z,2023-06-10T20:28:14Z,Windows,,Operating System,64.52.80.209
2,2022-07-06T14:59:30Z,2023-05-10T01:06:24Z,Ubuntu,,Operating System,64.52.80.209
3,2022-07-06T14:59:30Z,2023-05-10T01:06:24Z,SSH,2.0,Remote Access,64.52.80.209
4,2022-09-11T23:53:54Z,2023-05-10T01:06:24Z,OpenSSH,8.9p1,Remote Access,64.52.80.209
5,2023-03-01T05:30:13Z,2023-04-27T09:15:08Z,OpenSSH,8.4p1,Remote Access,64.52.80.209
6,2023-03-14T02:31:15Z,2023-04-15T02:56:18Z,Beego,,Framework,64.52.80.209
7,2023-03-14T02:31:15Z,2023-04-15T02:56:18Z,beegoServer:1.12.0,,Server,64.52.80.209
8,2023-03-04T02:02:33Z,2023-04-04T00:44:07Z,NPS,,Command and Control Server,64.52.80.209
9,2022-07-06T14:59:30Z,2022-11-22T02:39:35Z,OpenSSH,8.2p1,Remote Access,64.52.80.209


,firstSeenDateTime,lastSeenDateTime,name,version,category,host.id
0,2022-08-26T21:49:02Z,2023-06-14T22:49:24Z,Ubuntu,,Operating System,65.109.31.190
1,2022-07-31T00:31:40Z,2023-06-14T22:49:24Z,SSH,2.0,Remote Access,65.109.31.190
2,2022-08-26T21:49:02Z,2023-06-14T22:49:24Z,OpenSSH,8.9p1,Remote Access,65.109.31.190
3,2022-08-28T01:28:33Z,2023-06-14T16:33:57Z,nginx,1.18.0,Server,65.109.31.190
4,2022-08-02T08:58:50Z,2022-08-03T00:09:57Z,OpenSSH,8.4p1,Remote Access,65.109.31.190
5,2022-07-31T02:14:41Z,2022-08-01T23:34:21Z,Apache,,Server,65.109.31.190
6,2022-07-31T00:31:40Z,2022-08-01T00:27:56Z,OpenSSH,7.4,Remote Access,65.109.31.190
7,2022-07-31T21:21:33Z,2022-07-31T21:21:33Z,Exim Internet Mailer,4.95,Server,65.109.31.190
8,2022-07-31T19:35:52Z,2022-07-31T19:35:52Z,MySQL,,Data Store,65.109.31.190


,firstSeenDateTime,lastSeenDateTime,name,version,category,host.id
0,2020-09-07T19:17:42Z,2023-06-14T22:52:54Z,SSH,2.0,Remote Access,45.128.156.46
1,2021-08-17T12:34:52Z,2023-06-14T22:52:54Z,OpenSSH,8.4p1,Remote Access,45.128.156.46
2,2023-04-21T22:56:40Z,2023-06-14T02:29:54Z,ProFTPD,,Remote Access,45.128.156.46
3,2022-09-16T01:23:15Z,2023-01-27T08:39:05Z,OpenSSH,7.9p1,Remote Access,45.128.156.46
4,2021-08-17T12:34:52Z,2022-04-20T00:36:28Z,Ubuntu,,Operating System,45.128.156.46
5,2021-08-25T00:25:08Z,2022-04-19T11:31:48Z,nginx,1.18.0,Server,45.128.156.46
6,2022-01-25T10:05:32Z,2022-02-26T23:35:43Z,OpenSSH,7.4p1,Remote Access,45.128.156.46
7,2021-12-09T00:31:21Z,2022-01-23T13:24:56Z,vsftpd,3.0.2,Data Exchange,45.128.156.46
8,2021-12-09T00:31:21Z,2022-01-23T13:24:56Z,vsFTPd,3.0.2,Remote Access,45.128.156.46
9,2021-12-08T22:50:20Z,2022-01-23T09:53:35Z,OpenSSH,7.4,Remote Access,45.128.156.46


,firstSeenDateTime,lastSeenDateTime,name,version,category,host.id
0,2020-04-22T13:59:41Z,2023-06-14T22:50:13Z,SSH,2.0,Remote Access,84.252.94.184
1,2021-11-14T02:06:01Z,2023-06-14T22:50:13Z,Ubuntu,,Operating System,84.252.94.184
2,2021-11-14T02:06:01Z,2023-06-14T22:50:13Z,OpenSSH,8.2p1,Remote Access,84.252.94.184
3,2021-11-24T22:09:24Z,2022-01-15T10:35:31Z,mpm-itk,2.4.7-04,Server Module,84.252.94.184
4,2021-11-24T22:09:24Z,2022-01-15T10:35:31Z,OpenSSL,1.1.1h-freebsd,Server Module,84.252.94.184
5,2021-11-24T22:09:24Z,2022-01-15T10:35:31Z,Apache,2.4.46,Server,84.252.94.184
6,2021-11-24T22:09:24Z,2022-01-15T10:35:31Z,FreeBSD,,Server,84.252.94.184
7,2021-11-24T22:09:24Z,2022-01-15T10:35:31Z,FreeBSD,,Operating System,84.252.94.184
8,2021-03-02T03:15:57Z,2021-11-05T00:15:44Z,Windows Remote Desktop,,Remote Access,84.252.94.184
9,2021-03-02T03:15:57Z,2021-11-05T00:15:44Z,Windows,,Operating System,84.252.94.184


,firstSeenDateTime,lastSeenDateTime,name,version,category,host.id
0,2019-11-10T17:56:05Z,2023-06-14T22:54:47Z,FDC Servers,,Hosting Provider,192.240.116.106
1,2023-03-24T03:03:10Z,2023-06-14T22:54:47Z,SSH,2.0,Remote Access,192.240.116.106
2,2023-03-24T03:03:10Z,2023-06-14T22:54:47Z,OpenSSH,7.6p1,Remote Access,192.240.116.106
3,2019-11-22T18:43:13Z,2023-06-14T22:54:47Z,Ubuntu,,Operating System,192.240.116.106
4,2023-03-25T23:39:16Z,2023-06-13T06:08:36Z,nginx,1.14.0,Server,192.240.116.106
5,2019-11-13T02:58:40Z,2020-01-03T20:15:09Z,OpenSSH,6.6.1p1,Remote Access,192.240.116.106
6,2019-11-22T18:43:13Z,2020-01-03T12:08:01Z,Apache,2.4.7,Server,192.240.116.106
7,2019-11-22T18:43:13Z,2020-01-03T12:08:01Z,PHP,5.5.9-1ubuntu4.29,Framework,192.240.116.106
8,2019-11-22T18:43:13Z,2020-01-03T12:08:01Z,PHP,5.5.9-1ubuntu4.29,Server Module,192.240.116.106


,firstSeenDateTime,lastSeenDateTime,name,version,category,host.id
0,2022-11-09T02:10:38Z,2023-06-14T22:43:32Z,Ubuntu,,Operating System,5.255.100.206
1,2022-11-09T02:10:38Z,2023-06-14T22:43:32Z,OpenSSH,8.2p1,Remote Access,5.255.100.206
2,2021-11-19T12:40:01Z,2023-06-14T22:43:32Z,SSH,2.0,Remote Access,5.255.100.206
3,2022-11-09T22:03:57Z,2023-06-13T10:59:50Z,nginx,1.18.0,Server,5.255.100.206
4,2022-10-12T16:29:47Z,2022-10-12T16:29:47Z,OpenSSH,9.0,Remote Access,5.255.100.206
5,2022-03-08T02:38:44Z,2022-03-27T01:51:34Z,OpenSSH,8.0,Remote Access,5.255.100.206
6,2021-11-19T12:40:01Z,2022-03-02T11:41:53Z,OpenSSH,7.9p1,Remote Access,5.255.100.206
7,2020-09-23T00:51:54Z,2021-03-24T01:32:57Z,Squid,3.5.20,Server,5.255.100.206


C2 Check

This code block will query for the presence of the Web Component Category "Command and Control Server".  This could easily be customized to look for anything of interest to the investigation, this is simply an example for category based search.

In [107]:
pd.set_option('display.max_colwidth', 0)

suspect_iocs = ["64.52.80.63","5.252.177.180","193.149.189.224","64.190.113.172","64.52.80.209","65.109.31.190","45.128.156.46","84.252.94.184","192.240.116.106","5.255.100.206"]
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
for ip in suspect_iocs:

    services = f"https://graph.microsoft.com/beta/security/threatIntelligence/hosts/"+(ip)+"/components"
    responseS = requests.get(services, headers=headers)
    dataP = responseS.json()
    dataframeP = pd.json_normalize(dataP['value'])
    dataframeP.pop('id')
    cnc = (dataframeP[(dataframeP['category'] == 'Command and Control Server')])
    if not cnc.empty:
        display(cnc)
    



,firstSeenDateTime,lastSeenDateTime,name,version,category,host.id
8,2023-03-04T02:02:33Z,2023-04-04T00:44:07Z,NPS,,Command and Control Server,64.52.80.209


,firstSeenDateTime,lastSeenDateTime,name,version,category,host.id
21,2021-08-25T13:08:58Z,2021-11-10T07:14:18Z,Responder,,Command and Control Server,45.128.156.46
24,2021-09-22T00:23:54Z,2021-11-10T01:25:06Z,Cobalt Strike,,Command and Control Server,45.128.156.46


In [61]:
pd.set_option('display.max_colwidth', 0)

suspect_iocs = ["64.52.80.63","5.252.177.180","193.149.189.224","64.190.113.172","64.52.80.209","65.109.31.190","45.128.156.46","84.252.94.184","192.240.116.106","5.255.100.206","168.100.10.226","45.61.139.183","162.33.178.162","64.190.113.53","162.33.178.34","64.52.80.206"]
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
components1 = f"https://graph.microsoft.com/beta/security/threatIntelligence/hostComponents$search='cobalt'"
responseC = requests.get(components1, headers=headers)
cobalt = responseC.json()
dataframeCo = pd.json_normalize(cobalt)
#dataframeP.pop('id')
display(dataframeCo)

,error.code,error.message,error.innerError.date,error.innerError.request-id,error.innerError.client-request-id
0,BadRequest,Resource not found for the segment 'hostComponents$search='cobalt''.,2023-06-15T18:36:31,c538cbeb-0351-4414-945b-93be91ff86ba,c538cbeb-0351-4414-945b-93be91ff86ba


**PDNS Info** 

This snippet will allow you to pull PDNS hostnames from domains/IPs and run them through the reputation endpoint to learn what MDTI knows about them.  There is a commented line which will also look at the justification for the scoring.  You can uncomment that line to add in the additional information but be warned it could cause quite a lot of information to be returned if your array is large or the IP has many associated domains.

In [101]:
pd.set_option('display.max_colwidth', 0)

suspect_iocs = ["64.52.80.63","5.252.177.180","193.149.189.224","64.190.113.172","64.52.80.209","65.109.31.190","45.128.156.46","84.252.94.184"]

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
for ip in suspect_iocs:

    services = f"https://graph.microsoft.com/beta/security/threatIntelligence/hosts/"+(ip)+"/passivedns"
    responseS = requests.get(services, headers=headers)
    dataP = responseS.json()
    dataframeP = pd.json_normalize(dataP['value'])
    url = dataframeP['artifact.id'].values
    #url = dataframeP.drop(['id', 'firstSeenDateTime', 'lastSeenDateTime', 'collectedDateTime', 'recordType', 'parentHost.id', 'artifact.@odata.type'], axis=1).values
    

    for url1 in url:
        servicesR = f"https://graph.microsoft.com/beta/security/threatIntelligence/hosts/"+(url1)+'/reputation'
        responseR = requests.get(servicesR, headers=headers)
        dataS = responseR.json()
        dataframeS = pd.json_normalize(dataS)
        #dataframeSi = pd.json_normalize(dataS['rules'])
        display(dataframeS[['@odata.context','score', 'classification']])
        #display(dataframeS[['@odata.context','score', 'classification']], dataframeSi)

,@odata.context,score,classification
0,https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hosts('ijad.me')/reputation/$entity,100,malicious


,@odata.context,score,classification
0,https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hosts('fktb.me')/reputation/$entity,100,malicious


,@odata.context,score,classification
0,https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hosts('ewwk.link')/reputation/$entity,100,malicious


,@odata.context,score,classification
0,https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hosts('bxvr.me')/reputation/$entity,100,malicious


,@odata.context,score,classification
0,https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hosts('bjyh.me')/reputation/$entity,100,malicious


...

,@odata.context,score,classification
0,https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hosts('ffacco.top')/reputation/$entity,100,malicious


,@odata.context,score,classification
0,https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hosts('www.jgaact.top')/reputation/$entity,100,malicious


,@odata.context,score,classification
0,https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hosts('jgaact.top')/reputation/$entity,100,malicious


,@odata.context,score,classification
0,https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hosts('www.ffacco.top')/reputation/$entity,100,malicious


KeyError: "None of [Index(['@odata.context', 'score', 'classification'], dtype='object')] are in the [columns]"

***Trackers***

Trackers are embedded in web pages and can be google analytic IDs, Jarm Hashes, Facebook IDs, etc.  An example of where this is helpful is determining if a site may have rogue javascript injected or if a group of sites are using a custom javascript which can link the actors infrastructure together.

In [125]:
pd.set_option('display.max_colwidth', 0)

for ip in suspect_iocs:
    trackers = f"https://graph.microsoft.com/beta/security/threatIntelligence/hosts/"+(ip)+"/trackers?count=true"
    responseT = requests.get(trackers, headers=headers)
    dataT = responseT.json()
    dataframeT = pd.DataFrame(dataT['value'])
    #dataframeT = dataframeT.drop(['id'], axis=1)     
    display(dataframeT)

,id,firstSeenDateTime,lastSeenDateTime,kind,value,host
0,MjlkMjlkMDAwMjlkMjlkMjJjMjlkMjlkMjlkMjlkJCRKYXJtRnV6enlIYXNoJCQ2NC41Mi44MC42Mw==,2023-05-03T20:01:29Z,2023-06-13T22:57:51Z,JarmFuzzyHash,29d29d00029d29d22c29d29d29d29d,{'id': '64.52.80.63'}
1,MjlkMjlkMDAwMjlkMjlkMjJjMjlkMjlkMjlkMjlkYzJkZGNmZDIwM2QwNzFjNDViNGIwZmZlM2Q3YjRiODkkJEphcm1IYXNoJCQ2NC41Mi44MC42Mw==,2023-05-03T20:01:29Z,2023-06-13T22:57:51Z,JarmHash,29d29d00029d29d22c29d29d29d29dc2ddcfd203d071c45b4b0ffe3d7b4b89,{'id': '64.52.80.63'}
2,YzJkZGNmZDIwM2QwNzFjNDViNGIwZmZlM2Q3YjRiODkkJEphcm1FeHRlbnNpb25IYXNoJCQ2NC41Mi44MC42Mw==,2023-05-03T20:01:29Z,2023-06-13T22:57:51Z,JarmExtensionHash,c2ddcfd203d071c45b4b0ffe3d7b4b89,{'id': '64.52.80.63'}


,id,firstSeenDateTime,lastSeenDateTime,kind,value,host
0,MmFkMmFkMDAwMmFkMmFkMDAwMmFkMmFkMmFkMmFkJCRKYXJtRnV6enlIYXNoJCQ1LjI1Mi4xNzcuMTgw,2023-04-24T09:15:19Z,2023-06-13T16:02:51Z,JarmFuzzyHash,2ad2ad0002ad2ad0002ad2ad2ad2ad,{'id': '5.252.177.180'}
1,ZTFhM2MwZDdjYTZhZDgzODgwNTc5MjRiZTgzZGZjNmEkJEphcm1FeHRlbnNpb25IYXNoJCQ1LjI1Mi4xNzcuMTgw,2023-04-24T09:15:19Z,2023-06-13T16:02:51Z,JarmExtensionHash,e1a3c0d7ca6ad8388057924be83dfc6a,{'id': '5.252.177.180'}
2,MmFkMmFkMDAwMmFkMmFkMDAwMmFkMmFkMmFkMmFkZTFhM2MwZDdjYTZhZDgzODgwNTc5MjRiZTgzZGZjNmEkJEphcm1IYXNoJCQ1LjI1Mi4xNzcuMTgw,2023-04-24T09:15:19Z,2023-06-13T16:02:51Z,JarmHash,2ad2ad0002ad2ad0002ad2ad2ad2ade1a3c0d7ca6ad8388057924be83dfc6a,{'id': '5.252.177.180'}
3,OTQyNDgwM2E2NjJiMTI2YTc0OGNmNGY5MDcwN2EzM2MkJEphcm1FeHRlbnNpb25IYXNoJCQ1LjI1Mi4xNzcuMTgw,2022-09-23T16:35:00Z,2022-10-15T01:37:30Z,JarmExtensionHash,9424803a662b126a748cf4f90707a33c,{'id': '5.252.177.180'}
4,MDdkMTlkMTJkMjFkMjFkMDdjNDJkNDNkMDAwMDAwJCRKYXJtRnV6enlIYXNoJCQ1LjI1Mi4xNzcuMTgw,2022-09-23T16:35:00Z,2022-10-15T01:37:30Z,JarmFuzzyHash,07d19d12d21d21d07c42d43d000000,{'id': '5.252.177.180'}
5,MDdkMTlkMTJkMjFkMjFkMDdjNDJkNDNkMDAwMDAwOTQyNDgwM2E2NjJiMTI2YTc0OGNmNGY5MDcwN2EzM2MkJEphcm1IYXNoJCQ1LjI1Mi4xNzcuMTgw,2022-09-23T16:35:00Z,2022-10-15T01:37:30Z,JarmHash,07d19d12d21d21d07c42d43d0000009424803a662b126a748cf4f90707a33c,{'id': '5.252.177.180'}
6,MTVkMmFkMTZkMjlkMjlkMDAwMTVkMmFkMTVkMjlkZTg3ZTY1NjdkOTAxMzg4Nzk0Y2I2YTg3NWExOTI4YWEkJEphcm1IYXNoJCQ1LjI1Mi4xNzcuMTgw,2021-10-12T22:27:05Z,2022-02-05T19:42:20Z,JarmHash,15d2ad16d29d29d00015d2ad15d29de87e6567d901388794cb6a875a1928aa,{'id': '5.252.177.180'}
7,MTVkMmFkMTZkMjlkMjlkMDAwMTVkMmFkMTVkMjlkJCRKYXJtRnV6enlIYXNoJCQ1LjI1Mi4xNzcuMTgw,2021-10-12T22:27:05Z,2022-02-05T19:42:20Z,JarmFuzzyHash,15d2ad16d29d29d00015d2ad15d29d,{'id': '5.252.177.180'}
8,ZTg3ZTY1NjdkOTAxMzg4Nzk0Y2I2YTg3NWExOTI4YWEkJEphcm1FeHRlbnNpb25IYXNoJCQ1LjI1Mi4xNzcuMTgw,2021-10-12T22:27:05Z,2022-02-05T19:42:20Z,JarmExtensionHash,e87e6567d901388794cb6a875a1928aa,{'id': '5.252.177.180'}


,id,firstSeenDateTime,lastSeenDateTime,kind,value,host
0,MjhkMjhkMjhkMDAwMjhkMDAwNDJkNDJkMDAwMDAwJCRKYXJtRnV6enlIYXNoJCQxOTMuMTQ5LjE4OS4yMjQ=,2023-06-14T20:18:04Z,2023-06-14T20:18:04Z,JarmFuzzyHash,28d28d28d00028d00042d42d000000,{'id': '193.149.189.224'}
1,MjhkMjhkMjhkMDAwMjhkMDAwNDJkNDJkMDAwMDAwZDYzNzZmMWQ4ZmU2NDg3NzgwNmE4NWEwMDhjMWM3OTAkJEphcm1IYXNoJCQxOTMuMTQ5LjE4OS4yMjQ=,2023-06-14T20:18:04Z,2023-06-14T20:18:04Z,JarmHash,28d28d28d00028d00042d42d000000d6376f1d8fe64877806a85a008c1c790,{'id': '193.149.189.224'}
2,ZDYzNzZmMWQ4ZmU2NDg3NzgwNmE4NWEwMDhjMWM3OTAkJEphcm1FeHRlbnNpb25IYXNoJCQxOTMuMTQ5LjE4OS4yMjQ=,2023-06-14T20:18:04Z,2023-06-14T20:18:04Z,JarmExtensionHash,d6376f1d8fe64877806a85a008c1c790,{'id': '193.149.189.224'}


,id,firstSeenDateTime,lastSeenDateTime,kind,value,host
0,dWEtMjYyMDEwNjYzLTIkJEdvb2dsZUFuYWx5dGljc1RyYWNraW5nSWQkJDY0LjE5MC4xMTMuMTcy,2023-04-13T17:40:59Z,2023-06-14T18:41:15Z,GoogleAnalyticsTrackingId,ua-262010663-2,{'id': '64.190.113.172'}
1,dWEtMjYyMDEwNjYzJCRHb29nbGVBbmFseXRpY3NBY2NvdW50TnVtYmVyJCQ2NC4xOTAuMTEzLjE3Mg==,2023-04-13T17:40:59Z,2023-06-14T18:41:15Z,GoogleAnalyticsAccountNumber,ua-262010663,{'id': '64.190.113.172'}
2,MjFkMTlkMDAwMjFkMjFkMjFjMjFkMTlkMjFkMjFkJCRKYXJtRnV6enlIYXNoJCQ2NC4xOTAuMTEzLjE3Mg==,2023-03-11T17:09:01Z,2023-06-13T22:58:19Z,JarmFuzzyHash,21d19d00021d21d21c21d19d21d21d,{'id': '64.190.113.172'}
3,YTFhODE4YTk5OTg1ODg1NTQ0NWVjOGE4ZmRkMzhlYjUkJEphcm1FeHRlbnNpb25IYXNoJCQ2NC4xOTAuMTEzLjE3Mg==,2023-03-11T17:09:01Z,2023-06-13T22:58:19Z,JarmExtensionHash,a1a818a999858855445ec8a8fdd38eb5,{'id': '64.190.113.172'}
4,MjFkMTlkMDAwMjFkMjFkMjFjMjFkMTlkMjFkMjFkYTFhODE4YTk5OTg1ODg1NTQ0NWVjOGE4ZmRkMzhlYjUkJEphcm1IYXNoJCQ2NC4xOTAuMTEzLjE3Mg==,2023-03-11T17:09:01Z,2023-06-13T22:58:19Z,JarmHash,21d19d00021d21d21c21d19d21d21da1a818a999858855445ec8a8fdd38eb5,{'id': '64.190.113.172'}
5,bmVyYWdhbWVzJCRUZWxlZ3JhbVVzZXJJZCQkNjQuMTkwLjExMy4xNzI=,2022-10-26T23:56:34Z,2022-12-05T05:13:17Z,TelegramUserId,neragames,{'id': '64.190.113.172'}


,id,firstSeenDateTime,lastSeenDateTime,kind,value,host
0,MmFkMmFkMTZkMmFkMmFkMjJjNDJkNDJkMDAwMDAwNmYyNTQ5MDlhNzNiZjYyZjZiMjg1MDdlOWZiNDUxYjUkJEphcm1IYXNoJCQ2NC41Mi44MC4yMDk=,2022-10-08T19:09:43Z,2022-10-20T12:03:42Z,JarmHash,2ad2ad16d2ad2ad22c42d42d0000006f254909a73bf62f6b28507e9fb451b5,{'id': '64.52.80.209'}
1,NmYyNTQ5MDlhNzNiZjYyZjZiMjg1MDdlOWZiNDUxYjUkJEphcm1FeHRlbnNpb25IYXNoJCQ2NC41Mi44MC4yMDk=,2022-10-08T19:09:43Z,2022-10-20T12:03:42Z,JarmExtensionHash,6f254909a73bf62f6b28507e9fb451b5,{'id': '64.52.80.209'}
2,MmFkMmFkMTZkMmFkMmFkMjJjNDJkNDJkMDAwMDAwJCRKYXJtRnV6enlIYXNoJCQ2NC41Mi44MC4yMDk=,2022-10-08T19:09:43Z,2022-10-20T12:03:42Z,JarmFuzzyHash,2ad2ad16d2ad2ad22c42d42d000000,{'id': '64.52.80.209'}
3,MTVkM2ZkMTZkMjlkMjlkMDAwNDJkNDNkMDAwMDAwZWQxY2YzN2M5YTE2OWI0MTg4NmUyN2JhOGZhZDYwYjAkJEphcm1IYXNoJCQ2NC41Mi44MC4yMDk=,2022-09-17T14:54:08Z,2022-09-29T19:11:08Z,JarmHash,15d3fd16d29d29d00042d43d000000ed1cf37c9a169b41886e27ba8fad60b0,{'id': '64.52.80.209'}
4,MTVkM2ZkMTZkMjlkMjlkMDAwNDJkNDNkMDAwMDAwJCRKYXJtRnV6enlIYXNoJCQ2NC41Mi44MC4yMDk=,2022-09-17T14:54:08Z,2022-09-29T19:11:08Z,JarmFuzzyHash,15d3fd16d29d29d00042d43d000000,{'id': '64.52.80.209'}
5,ZWQxY2YzN2M5YTE2OWI0MTg4NmUyN2JhOGZhZDYwYjAkJEphcm1FeHRlbnNpb25IYXNoJCQ2NC41Mi44MC4yMDk=,2022-09-17T14:54:08Z,2022-09-29T19:11:08Z,JarmExtensionHash,ed1cf37c9a169b41886e27ba8fad60b0,{'id': '64.52.80.209'}


,id,firstSeenDateTime,lastSeenDateTime,kind,value,host
0,MmFkMmFkMDAwMmFkMmFkMDAwMmFkMmFkMmFkMmFkZTFhM2MwZDdjYTZhZDgzODgwNTc5MjRiZTgzZGZjNmEkJEphcm1IYXNoJCQ2NS4xMDkuMzEuMTkw,2022-08-29T02:29:19Z,2023-06-09T12:31:53Z,JarmHash,2ad2ad0002ad2ad0002ad2ad2ad2ade1a3c0d7ca6ad8388057924be83dfc6a,{'id': '65.109.31.190'}
1,MmFkMmFkMDAwMmFkMmFkMDAwMmFkMmFkMmFkMmFkJCRKYXJtRnV6enlIYXNoJCQ2NS4xMDkuMzEuMTkw,2022-08-29T02:29:19Z,2023-06-09T12:31:53Z,JarmFuzzyHash,2ad2ad0002ad2ad0002ad2ad2ad2ad,{'id': '65.109.31.190'}
2,ZTFhM2MwZDdjYTZhZDgzODgwNTc5MjRiZTgzZGZjNmEkJEphcm1FeHRlbnNpb25IYXNoJCQ2NS4xMDkuMzEuMTkw,2022-08-29T02:29:19Z,2023-06-09T12:31:53Z,JarmExtensionHash,e1a3c0d7ca6ad8388057924be83dfc6a,{'id': '65.109.31.190'}
3,MTVkM2ZkMTZkMjlkMjlkMDAwNDJkNDNkMDAwMDAwOWVjNjg2MjMzYTQzOThiZWEzMzRiYTVlNjJlMzRhMDEkJEphcm1IYXNoJCQ2NS4xMDkuMzEuMTkw,2022-07-31T09:52:10Z,2022-08-01T21:10:06Z,JarmHash,15d3fd16d29d29d00042d43d0000009ec686233a4398bea334ba5e62e34a01,{'id': '65.109.31.190'}
4,OWVjNjg2MjMzYTQzOThiZWEzMzRiYTVlNjJlMzRhMDEkJEphcm1FeHRlbnNpb25IYXNoJCQ2NS4xMDkuMzEuMTkw,2022-07-31T09:52:10Z,2022-08-01T21:10:06Z,JarmExtensionHash,9ec686233a4398bea334ba5e62e34a01,{'id': '65.109.31.190'}
5,MTVkM2ZkMTZkMjlkMjlkMDAwNDJkNDNkMDAwMDAwJCRKYXJtRnV6enlIYXNoJCQ2NS4xMDkuMzEuMTkw,2022-07-31T09:52:10Z,2022-08-01T21:10:06Z,JarmFuzzyHash,15d3fd16d29d29d00042d43d000000,{'id': '65.109.31.190'}


,id,firstSeenDateTime,lastSeenDateTime,kind,value,host
0,MWMxNDQ1YWU5OWI1NTc1YTNiNGI2MTkyZjQxYzAxNzQ0YzM2YWYxOGM4NDU1ZmI4NzQ3NzlmZDdjMGFlYjYxNiQkU1NIU2VydmVyUHVibGljS2V5JCQ0NS4xMjguMTU2LjQ2,2022-04-19T01:19:51Z,2022-04-20T00:36:28Z,SSHServerPublicKey,1c1445ae99b5575a3b4b6192f41c01744c36af18c8455fb874779fd7c0aeb616,{'id': '45.128.156.46'}
1,MmFkMmFkMDAwMmFkMmFkMDAwMmFkMmFkMmFkMmFkZTFhM2MwZDdjYTZhZDgzODgwNTc5MjRiZTgzZGZjNmEkJEphcm1IYXNoJCQ0NS4xMjguMTU2LjQ2,2022-03-10T21:56:05Z,2022-04-18T12:16:29Z,JarmHash,2ad2ad0002ad2ad0002ad2ad2ad2ade1a3c0d7ca6ad8388057924be83dfc6a,{'id': '45.128.156.46'}
2,MmFkMmFkMDAwMmFkMmFkMDAwMmFkMmFkMmFkMmFkJCRKYXJtRnV6enlIYXNoJCQ0NS4xMjguMTU2LjQ2,2022-03-10T21:56:05Z,2022-04-18T12:16:29Z,JarmFuzzyHash,2ad2ad0002ad2ad0002ad2ad2ad2ad,{'id': '45.128.156.46'}
3,ZTFhM2MwZDdjYTZhZDgzODgwNTc5MjRiZTgzZGZjNmEkJEphcm1FeHRlbnNpb25IYXNoJCQ0NS4xMjguMTU2LjQ2,2022-03-10T21:56:05Z,2022-04-18T12:16:29Z,JarmExtensionHash,e1a3c0d7ca6ad8388057924be83dfc6a,{'id': '45.128.156.46'}
4,ZWNmN2ZjYTdmYjA3MWNmNWE5NDk0OWZiY2MxNmVmNjkkJEphcm1FeHRlbnNpb25IYXNoJCQ0NS4xMjguMTU2LjQ2,2021-12-09T05:07:34Z,2022-01-20T22:27:07Z,JarmExtensionHash,ecf7fca7fb071cf5a94949fbcc16ef69,{'id': '45.128.156.46'}
5,MjlkMjlkMjBkMjlkMjlkMDAwMjlkMjlkMjlkMjlkZWNmN2ZjYTdmYjA3MWNmNWE5NDk0OWZiY2MxNmVmNjkkJEphcm1IYXNoJCQ0NS4xMjguMTU2LjQ2,2021-12-09T05:07:34Z,2022-01-20T22:27:07Z,JarmHash,29d29d20d29d29d00029d29d29d29decf7fca7fb071cf5a94949fbcc16ef69,{'id': '45.128.156.46'}
6,MjlkMjlkMjBkMjlkMjlkMDAwMjlkMjlkMjlkMjlkJCRKYXJtRnV6enlIYXNoJCQ0NS4xMjguMTU2LjQ2,2021-12-09T05:07:34Z,2022-01-20T22:27:07Z,JarmFuzzyHash,29d29d20d29d29d00029d29d29d29d,{'id': '45.128.156.46'}
7,YWQ5YmY1MWNjM2Y1YTFlMjllZWNiODFkMGM3YjA2ZWIkJEphcm1FeHRlbnNpb25IYXNoJCQ0NS4xMjguMTU2LjQ2,2021-08-25T12:59:00Z,2021-11-09T04:03:56Z,JarmExtensionHash,ad9bf51cc3f5a1e29eecb81d0c7b06eb,{'id': '45.128.156.46'}
8,MmFkMmFkMDAwMmFkMmFkMDAwNDJkNDJkMDAwMDAwJCRKYXJtRnV6enlIYXNoJCQ0NS4xMjguMTU2LjQ2,2021-08-25T12:59:00Z,2021-11-09T04:03:56Z,JarmFuzzyHash,2ad2ad0002ad2ad00042d42d000000,{'id': '45.128.156.46'}
9,MmFkMmFkMDAwMmFkMmFkMDAwNDJkNDJkMDAwMDAwYWQ5YmY1MWNjM2Y1YTFlMjllZWNiODFkMGM3YjA2ZWIkJEphcm1IYXNoJCQ0NS4xMjguMTU2LjQ2,2021-08-25T12:59:00Z,2021-11-09T04:03:56Z,JarmHash,2ad2ad0002ad2ad00042d42d000000ad9bf51cc3f5a1e29eecb81d0c7b06eb,{'id': '45.128.156.46'}


,id,firstSeenDateTime,lastSeenDateTime,kind,value,host
0,MDdkMTlkMTJkMjFkMjFkMDdjMDdkMTlkMDdkMjFkJCRKYXJtRnV6enlIYXNoJCQ4NC4yNTIuOTQuMTg0,2021-11-28T22:47:22Z,2022-01-13T21:48:03Z,JarmFuzzyHash,07d19d12d21d21d07c07d19d07d21d,{'id': '84.252.94.184'}
1,ZGJlYjIyYTdhMzZhMTliMmY1OTY0ZWM0Nzg0ODhmODMkJEphcm1FeHRlbnNpb25IYXNoJCQ4NC4yNTIuOTQuMTg0,2021-11-28T22:47:22Z,2022-01-13T21:48:03Z,JarmExtensionHash,dbeb22a7a36a19b2f5964ec478488f83,{'id': '84.252.94.184'}
2,MDdkMTlkMTJkMjFkMjFkMDdjMDdkMTlkMDdkMjFkZGJlYjIyYTdhMzZhMTliMmY1OTY0ZWM0Nzg0ODhmODMkJEphcm1IYXNoJCQ4NC4yNTIuOTQuMTg0,2021-11-28T22:47:22Z,2022-01-13T21:48:03Z,JarmHash,07d19d12d21d21d07c07d19d07d21ddbeb22a7a36a19b2f5964ec478488f83,{'id': '84.252.94.184'}


,id,firstSeenDateTime,lastSeenDateTime,kind,value,host
0,MmFkMmFkMDAwMmFkMmFkMjJjMmFkMmFkMmFkMmFkY2U3YTMyMWU0OTU2ZTgyOThiYTkxN2U5ZjJjMjI4NDkkJEphcm1IYXNoJCQxOTIuMjQwLjExNi4xMDY=,2023-03-28T02:11:39Z,2023-06-14T20:27:58Z,JarmHash,2ad2ad0002ad2ad22c2ad2ad2ad2adce7a321e4956e8298ba917e9f2c22849,{'id': '192.240.116.106'}
1,Y2U3YTMyMWU0OTU2ZTgyOThiYTkxN2U5ZjJjMjI4NDkkJEphcm1FeHRlbnNpb25IYXNoJCQxOTIuMjQwLjExNi4xMDY=,2023-03-28T02:11:39Z,2023-06-14T20:27:58Z,JarmExtensionHash,ce7a321e4956e8298ba917e9f2c22849,{'id': '192.240.116.106'}
2,MmFkMmFkMDAwMmFkMmFkMjJjMmFkMmFkMmFkMmFkJCRKYXJtRnV6enlIYXNoJCQxOTIuMjQwLjExNi4xMDY=,2023-03-28T02:11:39Z,2023-06-14T20:27:58Z,JarmFuzzyHash,2ad2ad0002ad2ad22c2ad2ad2ad2ad,{'id': '192.240.116.106'}


,id,firstSeenDateTime,lastSeenDateTime,kind,value,host
0,ZTFhM2MwZDdjYTZhZDgzODgwNTc5MjRiZTgzZGZjNmEkJEphcm1FeHRlbnNpb25IYXNoJCQ1LjI1NS4xMDAuMjA2,2022-11-09T09:21:52Z,2023-06-13T16:03:11Z,JarmExtensionHash,e1a3c0d7ca6ad8388057924be83dfc6a,{'id': '5.255.100.206'}
1,MmFkMmFkMDAwMmFkMmFkMDAwMmFkMmFkMmFkMmFkZTFhM2MwZDdjYTZhZDgzODgwNTc5MjRiZTgzZGZjNmEkJEphcm1IYXNoJCQ1LjI1NS4xMDAuMjA2,2022-11-09T09:21:52Z,2023-06-13T16:03:11Z,JarmHash,2ad2ad0002ad2ad0002ad2ad2ad2ade1a3c0d7ca6ad8388057924be83dfc6a,{'id': '5.255.100.206'}
2,MmFkMmFkMDAwMmFkMmFkMDAwMmFkMmFkMmFkMmFkJCRKYXJtRnV6enlIYXNoJCQ1LjI1NS4xMDAuMjA2,2022-11-09T09:21:52Z,2023-06-13T16:03:11Z,JarmFuzzyHash,2ad2ad0002ad2ad0002ad2ad2ad2ad,{'id': '5.255.100.206'}


In [130]:
pd.set_option('display.max_colwidth', 0)
tracker_id = "MjlkMjlkMDAwMjlkMjlkMjJjMjlkMjlkMjlkMjlkYzJkZGNmZDIwM2QwNzFjNDViNGIwZmZlM2Q3YjRiODkkJEphcm1IYXNoJCQ2NC41Mi44MC42Mw=="

tracker_lookup = f"https://graph.microsoft.com/beta/security/threatIntelligence/hostTrackers/"+(tracker_id)
responseTr = requests.get(tracker_lookup, headers=headers)
dataTr = responseTr.json()
dataframeTr = pd.json_normalize(dataTr)
    #dataframeT = dataframeT.drop(['id'], axis=1)     
display(dataframeTr)

,@odata.context,id,firstSeenDateTime,lastSeenDateTime,kind,value
0,https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hostTrackers/$entity,MjlkMjlkMDAwMjlkMjlkMjJjMjlkMjlkMjlkMjlkYzJkZGNmZDIwM2QwNzFjNDViNGIwZmZlM2Q3YjRiODkkJEphcm1IYXNoJCQ2NC41Mi44MC42Mw==,2023-05-03T20:01:29Z,2023-06-13T22:57:51Z,JarmHash,29d29d00029d29d22c29d29d29d29dc2ddcfd203d071c45b4b0ffe3d7b4b89
